In [2]:
# 최종 경로 반영

# 이미지 경로 변환 - any_approval DB 전자결재 문서용
# documents 테이블의 doc_body 컬럼 처리
# end_year로 연도별 필터링

import json
import os
import shutil
import re
from pathlib import Path
import pymysql
from sqlalchemy import create_engine

# ==================== DB 설정 ====================
USER = "root"
PASS = "1234"
HOST = "127.0.0.1"
PORT = 3306
DB = "any_approval"

# ==================== 설정 영역 (여기를 수정하세요) ====================

# 종료연도 (필터링용)
END_YEAR = 2020  # ✨ 2025, 2024, 2023 등

# 이미지 원본 경로들 - 여러 경로 추가 가능
BASE_IMAGE_PATHS = [
    # r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\namoeditor",
    # r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\daumeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\namoeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\daumeditor",
    # 더 추가하고 싶으면 여기에 계속 추가
]

# 이미지 복사본 저장 폴더 (연도별로 자동 생성)
OUTPUT_IMAGE_FOLDER = rf"C:\Users\LEEJUHWAN\Downloads\approval_{END_YEAR}_img"  # ✨ 연도 반영

# src에서 제거할 접두사 (이 부분을 제거하고 상대경로로 변경)
REMOVE_PREFIX = r"C:\Users\LEEJUHWAN\Downloads"

# ✨ NEW: src에 추가할 접두사
ADD_PREFIX = "/PMS_SITE-U7OI43JLDSMO/approval/"

# ======================================================================


def find_image_folder(folder_name):
    """여러 경로에서 이미지 폴더 찾기"""
    for base_path in BASE_IMAGE_PATHS:
        image_folder = Path(base_path) / folder_name
        if image_folder.exists():
            return image_folder
    return None


def process_single_document(data, doc_index):
    """하나의 document 데이터 처리"""
    
    # source_id에서 문서ID 추출
    source_id = data.get('source_id', '')
    if not source_id:
        print(f"⚠️ [{doc_index}] source_id 없음, 건너뜀")
        return data  # 원본 그대로 반환
    
    doc_id = source_id  # ✨ lstrip('_') 제거, 그대로 사용
    folder_name = f"apr{doc_id}"  # ✨ brd → apr
    
    # 여러 경로에서 이미지 폴더 찾기
    image_folder = find_image_folder(folder_name)
    
    if not image_folder:
        print(f"⚠️ [{doc_index}] 폴더 없음: {folder_name} (모든 경로 검색 완료)")
        return data  # 원본 그대로 반환
    
    # 복사본 저장할 문서별 폴더 생성
    output_doc_folder = Path(OUTPUT_IMAGE_FOLDER) / folder_name
    output_doc_folder.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📄 [{doc_index}] 문서ID: {doc_id}")
    print(f"   원본: {image_folder}")
    print(f"   복사: {output_doc_folder}")
    
    # HTML에서 img 태그 찾기
    html_content = data.get('doc_body', '')  # ✨ content → doc_body
    if not html_content:
        print(f"   ⚠️ doc_body 없음")
        return data
    
    img_pattern = r'<img[^>]+src="([^"]+)"[^>]*>'
    img_matches = list(re.finditer(img_pattern, html_content))
    
    if not img_matches:
        print(f"   ℹ️ 이미지 없음")
        return data
    
    print(f"   ✅ 이미지 {len(img_matches)}개 발견")
    
    # 파일 복사 & src 경로 변경
    new_html = html_content
    copied_count = 0
    changed_src_count = 0  # ✨ 추가: src 교체 성공 카운트
    
    for idx, match in enumerate(img_matches):
        old_src = match.group(1)  # src 속성 값
        old_img_tag = match.group(0)  # ✨ 전체 img 태그
        
        print(f"\n      [이미지 {idx}]")
        print(f"         원본 src: {old_src[:80]}...")
        
        # 원본 파일 (0, 1, 2...)
        old_file = image_folder / str(idx)
        
        # 복사본 파일 (.jpg 확장자 추가)
        new_file = output_doc_folder / f"{idx}.jpg"
        
        # 파일 복사
        if old_file.exists():
            shutil.copy2(old_file, new_file)
            copied_count += 1
            print(f"         ✅ 파일 복사: {old_file.name} → {new_file.name}")
            
            # src 경로 변경 (상대경로로)
            new_src = str(new_file).replace('\\', '/')
            if REMOVE_PREFIX:
                prefix_with_slash = REMOVE_PREFIX.replace('\\', '/')
                if new_src.startswith(prefix_with_slash):
                    new_src = new_src[len(prefix_with_slash):].lstrip('/')
            
            # ✨ NEW: 접두사 추가
            new_src = ADD_PREFIX + new_src
            
            print(f"         변경 src: {new_src}")
            
            # ✨ img 태그 전체를 교체 (src만 변경)
            new_img_tag = old_img_tag.replace(old_src, new_src)
            
            if old_img_tag in new_html:
                new_html = new_html.replace(old_img_tag, new_img_tag, 1)
                changed_src_count += 1
                print(f"         ✅ src 경로 교체 성공!")
            else:
                print(f"         ❌ 경고: HTML에서 img 태그를 찾을 수 없음!")
        else:
            print(f"         ⚠️ 파일 없음: {old_file.name} - src 공란 처리")
            # ✨ src를 빈 문자열로 변경
            new_img_tag = old_img_tag.replace(old_src, '')
            if old_img_tag in new_html:
                new_html = new_html.replace(old_img_tag, new_img_tag, 1)
                print(f"         → src를 공란으로 변경")
    
    print(f"\n   📊 파일 복사: {copied_count}/{len(img_matches)}개")
    print(f"   📊 src 경로 교체: {changed_src_count}/{len(img_matches)}개")  # ✨ 추가
    
    # 수정된 HTML 적용
    data['doc_body'] = new_html  # ✨ content → doc_body
    return data


def process_db(end_year):
    """DB에서 특정 end_year의 데이터만 읽고 처리 후 DB에 업데이트"""
    
    # 출력 폴더 생성
    os.makedirs(OUTPUT_IMAGE_FOLDER, exist_ok=True)
    
    # DB 연결
    print(f"📂 DB 연결 중...")
    print(f"📂 종료연도: {end_year}")  # ✨ 추가
    print(f"📂 검색할 이미지 경로: {len(BASE_IMAGE_PATHS)}개\n")
    
    connection = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASS,
        database=DB,
        port=PORT,
        charset='utf8mb4'
    )
    
    try:
        with connection.cursor(pymysql.cursors.DictCursor) as cursor:
            # 1. DB에서 특정 end_year의 데이터만 읽기
            query = "SELECT * FROM documents WHERE end_year = %s"  # ✨ WHERE 조건 추가
            cursor.execute(query, (end_year,))
            documents = cursor.fetchall()
            
            total_count = len(documents)
            print(f"✅ 총 {total_count}개의 documents 발견\n")
            
            # 2. 각 document 처리
            processed_count = 0
            
            for idx, doc in enumerate(documents, 1):
                try:
                    # 원본 doc_body 저장 (비교용)
                    original_content = doc.get('doc_body', '')
                    
                    # 기존 로직 그대로 사용
                    result = process_single_document(doc, idx)
                    
                    # doc_body가 변경된 경우만 DB 업데이트
                    if result['doc_body'] != original_content:
                        print(f"   🔄 doc_body 변경 감지! DB 업데이트 실행")  # ✨ 추가
                        update_query = """
                            UPDATE documents 
                            SET doc_body = %s 
                            WHERE source_id = %s AND end_year = %s
                        """  # ✨ WHERE 조건에 end_year도 추가
                        cursor.execute(update_query, (result['doc_body'], result['source_id'], end_year))
                        processed_count += 1
                    else:
                        print(f"   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵")  # ✨ 추가
                    
                except Exception as e:
                    print(f"❌ [Document {idx}] 처리 오류: {e}")
                    import traceback
                    traceback.print_exc()  # ✨ 추가: 상세 에러 출력
                    continue
            
            # 변경사항 커밋
            connection.commit()
            print(f"\n✅ DB 커밋 완료!")  # ✨ 추가
            
            print(f"\n{'='*60}")
            print(f"✅ 처리 완료!")
            print(f"   - 종료연도: {end_year}")  # ✨ 추가
            print(f"   - 처리된 documents: {processed_count}/{total_count}개")
            print(f"   - 복사된 이미지 위치: {OUTPUT_IMAGE_FOLDER}")
            print(f"   - DB 테이블: documents (doc_body 컬럼 업데이트)")
            print(f"{'='*60}")
    
    finally:
        connection.close()
        print("\n✅ DB 연결 종료")


# 실행
if __name__ == "__main__":
    process_db(END_YEAR)  # ✨ end_year 인자 추가

📂 DB 연결 중...
📂 종료연도: 2020
📂 검색할 이미지 경로: 2개

✅ 총 7704개의 documents 발견

⚠️ [1] 폴더 없음: apr2009497 (모든 경로 검색 완료)
   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵
⚠️ [2] 폴더 없음: apr2009503 (모든 경로 검색 완료)
   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵
⚠️ [3] 폴더 없음: apr2009500 (모든 경로 검색 완료)
   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵
⚠️ [4] 폴더 없음: apr2009502 (모든 경로 검색 완료)
   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵
⚠️ [5] 폴더 없음: apr2009499 (모든 경로 검색 완료)
   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵
⚠️ [6] 폴더 없음: apr2009501 (모든 경로 검색 완료)
   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵
⚠️ [7] 폴더 없음: apr2009498 (모든 경로 검색 완료)
   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵
⚠️ [8] 폴더 없음: apr2009507 (모든 경로 검색 완료)
   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵
⚠️ [9] 폴더 없음: apr2009504 (모든 경로 검색 완료)
   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵
⚠️ [10] 폴더 없음: apr2009505 (모든 경로 검색 완료)
   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵
⚠️ [11] 폴더 없음: apr2009508 (모든 경로 검색 완료)
   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵
⚠️ [12] 폴더 없음: apr2009506 (모든 경로 검색 완료)
   ⚠️ doc_body 변경 없음 - DB 업데이트 스킵
⚠️ [13] 폴더 없음: apr2009509 (모든 경로 검색 완료)
   ⚠️ doc_bo

In [1]:
# 이미지 경로 변환 - any_approval DB 전자결재 문서용
# documents 테이블의 doc_body 컬럼 처리
# end_year로 연도별 필터링

import json
import os
import shutil
import re
from pathlib import Path
import pymysql
from sqlalchemy import create_engine

# ==================== DB 설정 ====================
USER = "root"
PASS = "1234"
HOST = "127.0.0.1"
PORT = 3306
DB = "any_approval"

# ==================== 설정 영역 (여기를 수정하세요) ====================

# 종료연도 (필터링용)
END_YEAR = 2020  # ✨ 2025, 2024, 2023 등

# 이미지 원본 경로들 - 여러 경로 추가 가능
BASE_IMAGE_PATHS = [
    # r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\namoeditor",
    # r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\daumeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\namoeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\daumeditor",
    # 더 추가하고 싶으면 여기에 계속 추가
]

# 이미지 복사본 저장 폴더 (연도별로 자동 생성)
OUTPUT_IMAGE_FOLDER = rf"C:\Users\LEEJUHWAN\Downloads\approval_{END_YEAR}_img"  # ✨ 연도 반영

# src에서 제거할 접두사 (이 부분을 제거하고 상대경로로 변경)
REMOVE_PREFIX = r"C:\Users\LEEJUHWAN\Downloads"

# ======================================================================


def find_image_folder(folder_name):
    """여러 경로에서 이미지 폴더 찾기"""
    for base_path in BASE_IMAGE_PATHS:
        image_folder = Path(base_path) / folder_name
        if image_folder.exists():
            return image_folder
    return None


def process_single_document(data, doc_index):
    """하나의 document 데이터 처리"""
    
    # source_id에서 문서ID 추출
    source_id = data.get('source_id', '')
    if not source_id:
        print(f"⚠️ [{doc_index}] source_id 없음, 건너뜀")
        return data  # 원본 그대로 반환
    
    doc_id = source_id  # ✨ lstrip('_') 제거, 그대로 사용
    folder_name = f"apr{doc_id}"  # ✨ brd → apr
    
    # 여러 경로에서 이미지 폴더 찾기
    image_folder = find_image_folder(folder_name)
    
    if not image_folder:
        print(f"⚠️ [{doc_index}] 폴더 없음: {folder_name} (모든 경로 검색 완료)")
        return data  # 원본 그대로 반환
    
    # 복사본 저장할 문서별 폴더 생성
    output_doc_folder = Path(OUTPUT_IMAGE_FOLDER) / folder_name
    output_doc_folder.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📄 [{doc_index}] 문서ID: {doc_id}")
    print(f"   원본: {image_folder}")
    print(f"   복사: {output_doc_folder}")
    
    # HTML에서 img 태그 찾기
    html_content = data.get('doc_body', '')  # ✨ content → doc_body
    if not html_content:
        print(f"   ⚠️ doc_body 없음")
        return data
    
    img_pattern = r'<img[^>]+src="([^"]+)"[^>]*>'
    img_matches = list(re.finditer(img_pattern, html_content))
    
    if not img_matches:
        print(f"   ℹ️ 이미지 없음")
        return data
    
    print(f"   ✅ 이미지 {len(img_matches)}개 발견")
    
    # 파일 복사 & src 경로 변경
    new_html = html_content
    copied_count = 0
    
    for idx, match in enumerate(img_matches):
        old_src = match.group(1)
        
        # 원본 파일 (0, 1, 2...)
        old_file = image_folder / str(idx)
        
        # 복사본 파일 (.jpg 확장자 추가)
        new_file = output_doc_folder / f"{idx}.jpg"
        
        # 파일 복사
        if old_file.exists():
            shutil.copy2(old_file, new_file)
            copied_count += 1
            print(f"      ✅ {old_file.name} → {new_file.name}")
            
            # src 경로 변경 (상대경로로)
            new_src = str(new_file).replace('\\', '/')
            if REMOVE_PREFIX:
                prefix_with_slash = REMOVE_PREFIX.replace('\\', '/')
                if new_src.startswith(prefix_with_slash):
                    new_src = new_src[len(prefix_with_slash):].lstrip('/')
            
            new_html = new_html.replace(old_src, new_src, 1)
        else:
            print(f"      ⚠️ 파일 없음: {old_file.name} - src 공란 처리")
            # src를 빈 문자열로 변경
            new_html = new_html.replace(old_src, '', 1)
    
    print(f"   📊 복사 완료: {copied_count}/{len(img_matches)}개")
    
    # 수정된 HTML 적용
    data['doc_body'] = new_html  # ✨ content → doc_body
    return data


def process_db(end_year):
    """DB에서 특정 end_year의 데이터만 읽고 처리 후 DB에 업데이트"""
    
    # 출력 폴더 생성
    os.makedirs(OUTPUT_IMAGE_FOLDER, exist_ok=True)
    
    # DB 연결
    print(f"📂 DB 연결 중...")
    print(f"📂 종료연도: {end_year}")  # ✨ 추가
    print(f"📂 검색할 이미지 경로: {len(BASE_IMAGE_PATHS)}개\n")
    
    connection = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASS,
        database=DB,
        port=PORT,
        charset='utf8mb4'
    )
    
    try:
        with connection.cursor(pymysql.cursors.DictCursor) as cursor:
            # 1. DB에서 특정 end_year의 데이터만 읽기
            query = "SELECT * FROM documents WHERE end_year = %s"  # ✨ WHERE 조건 추가
            cursor.execute(query, (end_year,))
            documents = cursor.fetchall()
            
            total_count = len(documents)
            print(f"✅ 총 {total_count}개의 documents 발견\n")
            
            # 2. 각 document 처리
            processed_count = 0
            
            for idx, doc in enumerate(documents, 1):
                try:
                    # 원본 doc_body 저장 (비교용)
                    original_content = doc.get('doc_body', '')
                    
                    # 기존 로직 그대로 사용
                    result = process_single_document(doc, idx)
                    
                    # doc_body가 변경된 경우만 DB 업데이트
                    if result['doc_body'] != original_content:
                        update_query = """
                            UPDATE documents 
                            SET doc_body = %s 
                            WHERE source_id = %s AND end_year = %s
                        """  # ✨ WHERE 조건에 end_year도 추가
                        cursor.execute(update_query, (result['doc_body'], result['source_id'], end_year))
                        processed_count += 1
                    
                except Exception as e:
                    print(f"❌ [Document {idx}] 처리 오류: {e}")
                    continue
            
            # 변경사항 커밋
            connection.commit()
            
            print(f"\n{'='*60}")
            print(f"✅ 처리 완료!")
            print(f"   - 종료연도: {end_year}")  # ✨ 추가
            print(f"   - 처리된 documents: {processed_count}/{total_count}개")
            print(f"   - 복사된 이미지 위치: {OUTPUT_IMAGE_FOLDER}")
            print(f"   - DB 테이블: documents (doc_body 컬럼 업데이트)")
            print(f"{'='*60}")
    
    finally:
        connection.close()
        print("\n✅ DB 연결 종료")


# 실행
if __name__ == "__main__":
    process_db(END_YEAR)  # ✨ end_year 인자 추가

📂 DB 연결 중...
📂 종료연도: 2020
📂 검색할 이미지 경로: 2개

✅ 총 7704개의 documents 발견

⚠️ [1] 폴더 없음: apr2009497 (모든 경로 검색 완료)
⚠️ [2] 폴더 없음: apr2009503 (모든 경로 검색 완료)
⚠️ [3] 폴더 없음: apr2009500 (모든 경로 검색 완료)
⚠️ [4] 폴더 없음: apr2009502 (모든 경로 검색 완료)
⚠️ [5] 폴더 없음: apr2009499 (모든 경로 검색 완료)
⚠️ [6] 폴더 없음: apr2009501 (모든 경로 검색 완료)
⚠️ [7] 폴더 없음: apr2009498 (모든 경로 검색 완료)
⚠️ [8] 폴더 없음: apr2009507 (모든 경로 검색 완료)
⚠️ [9] 폴더 없음: apr2009504 (모든 경로 검색 완료)
⚠️ [10] 폴더 없음: apr2009505 (모든 경로 검색 완료)
⚠️ [11] 폴더 없음: apr2009508 (모든 경로 검색 완료)
⚠️ [12] 폴더 없음: apr2009506 (모든 경로 검색 완료)
⚠️ [13] 폴더 없음: apr2009509 (모든 경로 검색 완료)
⚠️ [14] 폴더 없음: apr2009510 (모든 경로 검색 완료)
⚠️ [15] 폴더 없음: apr2009511 (모든 경로 검색 완료)
⚠️ [16] 폴더 없음: apr2009512 (모든 경로 검색 완료)
⚠️ [17] 폴더 없음: apr2009513 (모든 경로 검색 완료)
⚠️ [18] 폴더 없음: apr2009514 (모든 경로 검색 완료)
⚠️ [19] 폴더 없음: apr2009516 (모든 경로 검색 완료)
⚠️ [20] 폴더 없음: apr2009515 (모든 경로 검색 완료)
⚠️ [21] 폴더 없음: apr2009519 (모든 경로 검색 완료)
⚠️ [22] 폴더 없음: apr2009518 (모든 경로 검색 완료)
⚠️ [23] 폴더 없음: apr2009517 (모든 경로 검색 완료)
⚠️ [24] 폴더 없음: apr20